## RAG example with Langchain, Milvus, and OLLAMA
Requirements:
- A Milvus instance, either standalone or cluster.
- Connection credentials to Milvus must be available as environment variables: MILVUS_USERNAME and MILVUS_PASSWORD.
- An OLLAMA inference endpoint.

### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0 openai==1.13.3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Milvus

#### Bases parameters, Inference server and Milvus info

In [3]:
os.environ["OPENAI_API_KEY"] = "EMPTY"
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [4]:
# Replace values according to your Milvus deployment
### ocp route
INFERENCE_SERVER_URL ="http://ollama:11434"
MODEL_NAME = "mistral"
MILVUS_HOST = "vectordb-milvus"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "redhat_notes"

#### Initialize the connection

In [5]:
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



<All keys matched successfully>


#### Initialize query chain

In [6]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant named SnemeisBot answering questions.
You will be given a question you need to answer, and a context about everything a sales team wrote down about its customers to provide you with information. You must answer the question based as much as possible on this context.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, don't share false information.
<</SYS>>

Context: 
{context}

Question: {question} [/INST]
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = Ollama(base_url=INFERENCE_SERVER_URL, model=MODEL_NAME)

qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 1}
            ),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT, "verbose": True},
        return_source_documents=True
        )

os.environ["TOKENIZERS_PARALLELISM"] = "false"

#### Query example

In [7]:
question = "Who is your mother?"
result = qa_chain.invoke({"query": question})

#### Retrieve source

In [8]:
type(result)

dict

In [9]:
result

{'query': 'Who is your mother?',
 'result': ' I\'m sorry, the question "Who is your mother?" does not make any sense in relation to the provided context. The context discusses various business-related information about different individuals and organizations, but it does not provide any information about a "mother" figure or entity. If you meant to ask about a specific term or concept within the context, please clarify and I\'ll be happy to help answer any question related to that.',
 'source_documents': [Document(page_content='Bob Brandenstein: used to make certifications at PLM but we did not speak to him for a long time\n\n\n\nMOM/MES (Opcenter within Manufacturing)\n\n\n\nHead is ZVI Feuer in Israel\n\nMany stakeholders in the US\n\nItalien Architects developed modular MOM on OpenShift which runs in some factories in Germany\n\nChance to get a blueprint for MOM on OpenShift on premise\n\nRaymond Kok might connect\n\nKey product is Opcenter: Opcenter is a holistic Manufacturing Oper

In [10]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)

./snemeis-docx/ US focus @ Siemens _.docx
